## Homework 01-intro

### Q1. Running Elastic

In [2]:
!curl localhost:9200

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 9200 after 2225 ms: Couldn't connect to server


### Q2. Indexing the data

In [4]:
from elasticsearch import Elasticsearch

In [4]:
help(Elasticsearch)

Help on class Elasticsearch in module elasticsearch:

class Elasticsearch(elasticsearch._sync.client._base.BaseClient)
 |  Elasticsearch(hosts: Union[str, Sequence[Union[str, Mapping[str, Union[str, int]], elastic_transport.NodeConfig]], NoneType] = None, *, cloud_id: Optional[str] = None, api_key: Union[str, Tuple[str, str], NoneType] = None, basic_auth: Union[str, Tuple[str, str], NoneType] = None, bearer_auth: Optional[str] = None, opaque_id: Optional[str] = None, headers: Union[elastic_transport.client_utils.DefaultType, Mapping[str, str]] = <DEFAULT>, connections_per_node: Union[elastic_transport.client_utils.DefaultType, int] = <DEFAULT>, http_compress: Union[elastic_transport.client_utils.DefaultType, bool] = <DEFAULT>, verify_certs: Union[elastic_transport.client_utils.DefaultType, bool] = <DEFAULT>, ca_certs: Union[elastic_transport.client_utils.DefaultType, str] = <DEFAULT>, client_cert: Union[elastic_transport.client_utils.DefaultType, str] = <DEFAULT>, client_key: Union[ela

  :param index: Comma-separated list of data streams, indices, and aliases to search.
 |          Supports wildcards (`*`). To search all data streams or indices, omit this
 |          parameter or use `*` or `_all`.

### Q3

In [5]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
es_client = Elasticsearch('http://localhost:9200') 

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

c:\Users\Aussar\AppData\Local\pypoetry\Cache\virtualenvs\llm-zoomcamp-2024-K4UpOaut-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:47<00:00, 19.82it/s]


In [10]:
query="How do I execute a command in a running docker container?"

In [11]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [12]:
elastic_search(query)

[75.54128, 43.922554, 38.684105, 38.33403, 35.94081]

### Q4. Filtering

In [13]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [14]:
elastic_search(query)

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

### Q5. Building a prompt

In [15]:
final_context = """"""

context_template = """
Q: {question}
A: {text}
""".strip()

for n in elastic_search(query):
    question = n['question']
    text = n['text']
    context_template = context_template.format(question=question, text=text)
    final_context = final_context + f"{context_template}\n\n"

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".format(question="How do I execute a command in a running docker container?", context=final_context)

In [17]:
len(prompt_template)

1326

### Q6. Tokens

In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
len(encoding.encode(prompt_template))

299